In [1]:
import numpy as np
import pandas as pd

In [2]:
training_data = pd.read_csv("train.csv")
testing_data = pd.read_csv("test.csv")

In [3]:
print(training_data.head())

   Id   Open Date        City  City Group Type  P1   P2   P3   P4  P5  \
0   0  07/17/1999    İstanbul  Big Cities   IL   4  5.0  4.0  4.0   2   
1   1  02/14/2008      Ankara  Big Cities   FC   4  5.0  4.0  4.0   1   
2   2  03/09/2013  Diyarbakır       Other   IL   2  4.0  2.0  5.0   2   
3   3  02/02/2012       Tokat       Other   IL   6  4.5  6.0  6.0   4   
4   4  05/09/2009   Gaziantep       Other   IL   3  4.0  3.0  4.0   2   

     ...      P29  P30  P31  P32  P33  P34  P35  P36  P37    revenue  
0    ...      3.0    5    3    4    5    5    4    3    4  5653753.0  
1    ...      3.0    0    0    0    0    0    0    0    0  6923131.0  
2    ...      3.0    0    0    0    0    0    0    0    0  2055379.0  
3    ...      7.5   25   12   10    6   18   12   12    6  2675511.0  
4    ...      3.0    5    1    3    2    3    4    3    3  4316715.0  

[5 rows x 43 columns]


In [4]:
ids = np.array(testing_data["Id"])

In [5]:
training_data = training_data.drop("Id",axis = 1)
testing_data = testing_data.drop("Id",axis = 1)

In [6]:
current_date = pd.to_datetime("today")
current_year = current_date.year

training_data["Open Date"] = pd.to_datetime(training_data["Open Date"])
testing_data["Open Date"] = pd.to_datetime(testing_data["Open Date"])

train_years = training_data["Open Date"].dt.year
test_years = testing_data["Open Date"].dt.year

training_data["Years Open"] = current_year - train_years
testing_data["Years Open"] = current_year - test_years

training_data = training_data.drop("Open Date",axis = 1)
testing_data = testing_data.drop("Open Date",axis = 1)

In [7]:
training_data = training_data.drop("City",axis = 1)
testing_data = testing_data.drop("City",axis = 1)

In [8]:
city_group_train = pd.get_dummies(training_data["City Group"])
city_group_test = pd.get_dummies(testing_data["City Group"])

training_data = training_data.join(city_group_train)
testing_data = testing_data.join(city_group_test)

training_data = training_data.drop("City Group",axis = 1)
testing_data = testing_data.drop("City Group",axis =1)

In [9]:
print(training_data.head())

  Type  P1   P2   P3   P4  P5  P6  P7  P8  P9  ...    P32  P33  P34  P35  P36  \
0   IL   4  5.0  4.0  4.0   2   2   5   4   5  ...      4    5    5    4    3   
1   FC   4  5.0  4.0  4.0   1   2   5   5   5  ...      0    0    0    0    0   
2   IL   2  4.0  2.0  5.0   2   3   5   5   5  ...      0    0    0    0    0   
3   IL   6  4.5  6.0  6.0   4   4  10   8  10  ...     10    6   18   12   12   
4   IL   3  4.0  3.0  4.0   2   2   5   5   5  ...      3    2    3    4    3   

   P37    revenue  Years Open  Big Cities  Other  
0    4  5653753.0          19           1      0  
1    0  6923131.0          10           1      0  
2    0  2055379.0           5           0      1  
3    6  2675511.0           6           0      1  
4    3  4316715.0           9           0      1  

[5 rows x 42 columns]


In [10]:
city_type_train = pd.get_dummies(training_data["Type"])
city_type_test = pd.get_dummies(testing_data["Type"])

training_data = training_data.join(city_type_train)
testing_data = testing_data.join(city_type_test)

training_data = training_data.drop("Type",axis = 1)
testing_data = testing_data.drop("Type",axis =1)

In [11]:
print(training_data.head())
print(testing_data.head())

   P1   P2   P3   P4  P5  P6  P7  P8  P9  P10 ...  P35  P36  P37    revenue  \
0   4  5.0  4.0  4.0   2   2   5   4   5    5 ...    4    3    4  5653753.0   
1   4  5.0  4.0  4.0   1   2   5   5   5    5 ...    0    0    0  6923131.0   
2   2  4.0  2.0  5.0   2   3   5   5   5    5 ...    0    0    0  2055379.0   
3   6  4.5  6.0  6.0   4   4  10   8  10   10 ...   12   12    6  2675511.0   
4   3  4.0  3.0  4.0   2   2   5   5   5    5 ...    4    3    3  4316715.0   

   Years Open  Big Cities  Other  DT  FC  IL  
0          19           1      0   0   0   1  
1          10           1      0   0   1   0  
2           5           0      1   0   0   1  
3           6           0      1   0   0   1  
4           9           0      1   0   0   1  

[5 rows x 44 columns]
   P1   P2   P3   P4  P5  P6  P7  P8  P9  P10 ...  P35  P36  P37  Years Open  \
0   1  4.0  4.0  4.0   1   2   5   4   5    5 ...    0    0    0           7   
1   3  4.0  4.0  4.0   2   2   5   3   4    4 ...    0    0 

In [12]:
testing_data = testing_data.drop("MB",axis = 1)

In [13]:
training_data.to_csv("processed_train_data.csv")
testing_data.to_csv("processed_test_data.csv")

In [14]:
training_targets = training_data["revenue"]
training_data = training_data.drop("revenue",axis = 1)

In [15]:
print(training_data.shape)
print(testing_data.shape)

(137, 43)
(100000, 43)


In [16]:
from sklearn.ensemble import RandomForestRegressor
model = RandomForestRegressor(n_estimators = 1000,n_jobs = -1)
fitter = model.fit(training_data,training_targets)
predictions = model.predict(testing_data)

In [17]:
print(predictions[:10])

[ 4027179.176  3464288.022  3648663.578  3616550.257  4632124.816
  4156384.247  4069865.854  3983924.206  4991612.58   3521215.481]


In [18]:
final_preds = pd.DataFrame({"id": ids,"Prediction": predictions})

In [19]:
print(final_preds.head())

    Prediction  id
0  4027179.176   0
1  3464288.022   1
2  3648663.578   2
3  3616550.257   3
4  4632124.816   4


In [20]:
final_preds.to_csv("revenue_prediction.csv",index = False)

In [21]:
import pickle
filename = "random_forest_model.sav"
pickle.dump(model,open(filename,"wb"))